In [ ]:
DATA = {
    'client_id':'',
    'client_secret':''
}

BASE_URL = 'http://localhost:19999'

In [ ]:
# Login
import requests 

response = requests.post(BASE_URL + '/api/4.0/login',data=DATA)
print(response.status_code)
access_token = response.json()['access_token']
headers= {
    'Authorization': 'token ' + access_token,
    'x-looker-appid': 'marketplace_extension_hackathon::hackathon'
}
response = requests.get(
    BASE_URL + '/api/4.0/artifact/namespaces',
    headers=headers)

print(response.json())

In [ ]:
import json
TABLE_NAME = 'User'

response = requests.get(
    BASE_URL + '/api/4.0/artifact/hackathon/search',
    data={'key': TABLE_NAME + '%'},
    headers=headers)

print(response.json())

In [ ]:
# Migrate all data 
# Download each google sheet as csv
# Expects directory/naming to be: csv/hackathons.csv
import json
import csv

def toValues(path):
    with open(path, newline='') as csvfile:
        reader = csv.reader(x.replace('\0', '') for x in csvfile)
        header = next(reader)
        values = []
        for row in reader:
            value = {}
            for index, v in enumerate(row):
                value[header[index]] = v
            values.append(value)
        return values
    
def toArtifact(key, value):
    artifact = {}
    artifact['key'] = key
    artifact['content_type'] = 'application/json'
    artifact['value'] = json.dumps(value)
    return artifact
    
artifacts = []

for v in toValues('csv/hackathons.csv'):
    key = 'Hackathon:' + v['_id']
    del v['_id'] 
    artifacts.append(toArtifact(key, v))

for v in toValues('csv/judgings.csv'):
    key = 'Judging:' + v['_id']
    del v['_id'] 
    v['user_id'] = 'User:' + v['user_id']
    v['project_id'] = 'Project:' + v['project_id']
    artifacts.append(toArtifact(key, v))

for v in toValues('csv/projects.csv'):
    key = 'Project:' + v['_id']
    del v['_id']
    v['_user_id'] = 'User:' + v['_user_id']
    v['_hackathon_id'] = 'Hackathon:' + v['_hackathon_id']
    v['technologies'] = ['Technology:' + x for x in v['technologies'].split(',')]
    artifacts.append(toArtifact(key, v))

for v in toValues('csv/registrations.csv'):
    key = 'Registration:' + v['_id']
    del v['_id'] 
    v['_user_id'] = 'User:' + v['_user_id']
    v['hackathon_id'] = 'Hackathon:' + v['hackathon_id']
    artifacts.append(toArtifact(key, v))

for v in toValues('csv/team_members.csv'):
    key = 'TeamMember:' + v['_id']
    del v['_id'] 
    v['user_id'] = 'User:' + v['user_id']
    v['project_id'] = 'Project:' + v['project_id']
    artifacts.append(toArtifact(key, v))

for v in toValues('csv/technologies.csv'):
    key = 'Technology:' + v['_id']
    del v['_id'] 
    artifacts.append(toArtifact(key, v))

for v in toValues('csv/users.csv'):
    key = 'User:' + v['_id']
    v['looker_id'] = '' + v['_id']
    del v['_id'] 
    artifacts.append(toArtifact(key, v))

response = requests.put(BASE_URL + '/api/4.0/artifacts/hackathon', headers=headers, json=artifacts) 

print(response.status_code)


In [ ]:
# TO PURGE ALL TABLES. BE CAREFUL
response = requests.delete(BASE_URL + '/api/4.0/artifact/Hackathon/purge',headers=headers)
print(response.status_code)

response = requests.delete(BASE_URL + '/api/4.0/artifact/Judging/purge',headers=headers)
print(response.status_code)

response = requests.delete(BASE_URL + '/api/4.0/artifact/Project/purge',headers=headers)
print(response.status_code)

response = requests.delete(BASE_URL + '/api/4.0/artifact/Registration/purge',headers=headers)
print(response.status_code)

response = requests.delete(BASE_URL + '/api/4.0/artifact/TeamMember/purge',headers=headers)
print(response.status_code)

response = requests.delete(BASE_URL + '/api/4.0/artifact/Technology/purge',headers=headers)
print(response.status_code)

response = requests.delete(BASE_URL + '/api/4.0/artifact/User/purge',headers=headers)
print(response.status_code)
